# 2020 USA Election: Vote Percentages by State

In [1]:
import pandas as pd
import time
from datetime import datetime
import plotly_express as px
from plotly.offline import init_notebook_mode
init_notebook_mode(connected=True)
todays_date = datetime.fromtimestamp(time.time()).strftime("%Y-%m-%d")

In [2]:
df_president_county = pd.read_csv('/kaggle/input/us-election-2020/president_county_candidate.csv')
democrat = df_president_county['party'] == 'DEM'
republican = df_president_county['party'] == 'REP'
df_president_county = df_president_county[democrat|republican]
df_president_county = df_president_county.groupby(['state','party']).sum()
df_president_county = df_president_county.unstack()
df_president_county.columns = ['DEM','REP']
df_president_county = df_president_county.reset_index()

state_codes = pd.read_csv('/kaggle/input/latitude-and-longitude-for-every-country-and-state/world_country_and_usa_states_latitude_and_longitude_values.csv')
state_codes = state_codes[['usa_state','usa_state_code']]
df_president_county = df_president_county.merge(state_codes, left_on='state', right_on='usa_state')
df_president_county['percent_democrat'] = df_president_county['DEM']*100/(df_president_county['REP']+df_president_county['DEM'])

In [3]:
fig = px.choropleth(df_president_county, 
                    locations="usa_state_code", 
                    color = "percent_democrat",
                    locationmode = 'USA-states', 
                    hover_name="state",
                    range_color=[25,75],
                    color_continuous_scale = 'RdBu',#blues
                    scope="usa",
                    title='2020 USA Election: Percent of Population Voting for the Democratic Party (as of '+todays_date+')')
fig.show()

In [4]:
df_vote_counts = pd.read_csv('/kaggle/input/us-election-2020/president_county_candidate.csv')
democrat = df_vote_counts['party'] == 'DEM'
republican = df_vote_counts['party'] == 'REP'
df_vote_counts = df_vote_counts[democrat|republican]
df_vote_counts = pd.DataFrame(df_vote_counts.groupby('party')['votes'].sum())
fig = px.bar(df_vote_counts,title='2020 USA Election: Number of Votes per Party (Democratic vs Republican)')
fig.show()

In [5]:
df_vote_percent = df_vote_counts.copy()
df_vote_percent['votes'] = df_vote_percent['votes']*100/df_vote_percent['votes'].sum()
fig = px.bar(df_vote_percent,title='2020 USA Election: Percent of Votes per Party (Democratic vs Republican)')
fig.update_yaxes(range=[45, 55])
fig.show()

In [6]:
df_president = pd.read_csv('/kaggle/input/us-election-2020/president_state.csv')
state_codes = pd.read_csv('/kaggle/input/latitude-and-longitude-for-every-country-and-state/world_country_and_usa_states_latitude_and_longitude_values.csv')
state_codes = state_codes[['usa_state','usa_state_code']]
df_president = df_president.merge(state_codes, left_on='state', right_on='usa_state')

In [7]:
fig = px.choropleth(df_president, 
                    locations="usa_state_code", 
                    color="votes", 
                    locationmode = 'USA-states', 
                    hover_name="state",
                    range_color=[0,10000000],scope="usa",
                    title='2020 USA Election: Presidential Votes Counts (as of '+todays_date+')')
fig.show()

In [8]:
#!mkdir /kaggle/working/docker/
#!pip freeze > '/kaggle/working/docker/requirements.txt'
print('This notebook makes use of \nthe following Python libraries:\n')
print('pandas:',pd.__version__)
import plotly_express
print('plotly express:',plotly_express.__version__)
df_president_county.to_csv('democratic_vs_republican_votes_by_usa_state_2020.csv',index=False)
df_president.to_csv('number_of_voters_per_usa_state_2020.csv',index=False)
df_vote_counts.to_csv('vote_counts_democratic_vs_republican_for_entire_country.csv',index=False)
df_vote_percent.to_csv('vote_percentages_democratic_vs_republican_for_entire_country.csv',index=False)

This notebook makes use of 
the following Python libraries:

pandas: 1.1.3
plotly express: 0.4.1
